In [5]:
from db_connect import db_connect
import psycopg2
import os
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
import pandas as pd

In [ ]:
# DB 연결
cur,conn=db_connect()

# 데이터 프레임으로 불러오기
df= pd.read_sql("SELECT * FROM cafe WHERE congestion_score IS NOT NULL",conn)
df.replace([float('inf'), float('-inf')],pd.NA, inplace=True)

cur.close()
conn.close()

# region 컬럼 추가
df['region']=df['address'].apply(lambda x: x.split()[1] if x and len(x.split()) > 1 else None)

DB connect 성공


C:\Users\hyunsu\AppData\Local\Temp\ipykernel_22496\3276972236.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df= pd.read_sql("SELECT * FROM cafe WHERE congestion_score IS NOT NULL",conn)


In [15]:
# 전처리
df = df.dropna(subset=['review_count','congestion_score'])

le_region = LabelEncoder()
df['region_encoded'] = le_region.fit_transform(df['region'])

x = df[['review_count', 'region_encoded']]
y = df[['congestion_score']]

# 모델 학습
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=42)
model = RandomForestRegressor()
model.fit(x_train,y_train)

# 모델 평가
y_pred = model.predict(x_test)
print("MSE:",mean_squared_error(y_test,y_pred))

MSE: 0.01978695652173911


C:\Users\hyunsu\AppData\Local\Temp\ipykernel_22496\2955728502.py:13: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(x_train,y_train)


In [ ]:
df[df['region']=='강남구'].sort_values('review_count',ascending=False).head(5)

region
강남구     167
마포구     118
용산구     112
서대문구     95
종로구      90
성동구      54
광진구      53
중구        1
Name: count, dtype: int64